In [15]:
import illustris_python as il
import numpy as np
import sys
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

sys.path.append('..')

from shared.utilities import get_redshift_dictionary

base_path = "/Users/users/nastase/PROJECT/"
output_path = "/Users/users/nastase/PROJECT/DATA/velocity_files_for_dtfe/"

def load_data(snapshot_number):
    data = il.snapshot.loadSubset(base_path, snapshot_number, 'dm', ["Coordinates", "Velocities", "ParticleIDs"])
    data["Masses"] = np.ones(len(data["Coordinates"]))

    
    return data

def process_snapshot(snapshot_number):
    print(f"Start processing {snapshot_number}")
    data = load_data(snapshot_number)
    convert_data_to_tex_even_faster(data, snapshot_number, output_path)
    
    print(f"Finish processing {snapshot_number}")

def convert_data_to_tex_even_faster(data, snapshot_number, path):
    
    particle_number = data["count"]
    coordinates = data["Coordinates"]
    velocities = data["Velocities"]
    particleIDs = data["ParticleIDs"]
    masses = data["Masses"]
    
    box_min = 0
    box_max = 75000
    
    filename = f"{path}/file_vel_ids_{snapshot_number}.txt"
    header = f"{particle_number}\n{box_min:.6f} {box_max:.6f} {box_min:.6f} {box_max:.6f} {box_min:.6f} {box_max:.6f}\n"
    all_data = np.hstack((coordinates, velocities,masses.reshape(-1, 1), particleIDs.reshape(-1,1)))

    
    # Convert all data to formatted strings
    lines = "\n".join(f"{x:.6f} {y:.6f} {z:.6f} {vx:.6f} {vy:.6f} {vz:.6f} {m:.6f} {ids:.6f}" for x, y, z, vx, vy, vz, m, ids in all_data ) 
    
    with open(filename, 'w') as f:
        f.write(header)
        f.write(lines)
        f.write('\n')  # Add a newline at the end     

In [7]:
data135 = load_data(135)

In [16]:
convert_data_to_tex_even_faster(data135, 135, output_path)

In [10]:
data135

{'count': 94196375,
 'Coordinates': array([[  860.2786  , 26320.94    , 18281.758   ],
        [  862.8968  , 26320.16    , 18283.193   ],
        [  859.6459  , 26324.168   , 18280.25    ],
        ...,
        [60536.855   , 49059.383   , 56893.633   ],
        [65776.43    , 61606.74    , 46854.918   ],
        [64010.445   , 61434.027   ,    70.096725]], dtype=float32),
 'Velocities': array([[  37.119167, -126.95646 , -137.56683 ],
        [ -62.20938 , -280.38562 , -148.18048 ],
        [  79.16669 , -229.4381  ,  -60.910133],
        ...,
        [  88.98358 ,  148.74493 , -175.3597  ],
        [ 242.84515 ,   16.739655,  113.39123 ],
        [ 174.96793 ,   17.906925,   -8.918241]], dtype=float32),
 'ParticleIDs': array([88038007, 88173007,  3638407, ..., 94196369, 94196373, 94196375],
       dtype=uint64),
 'Masses': array([1., 1., 1., ..., 1., 1., 1.])}

In [5]:
#     dictionary = get_redshift_dictionary()
#     dictionary['snapshots']
snapshot_numbers = [135, 133]

#     ##this is for more I/O bound tasks    
#     with ThreadPoolExecutor() as executor:
#         executor.map(process_snapshot, snapshot_numbers)
    
    #this is for more other (MY KIND) kinds of tasks? 
with ProcessPoolExecutor() as executor:
    executor.map(process_snapshot, snapshot_numbers)


Start processing 135Start processing 133

